## Dimensional Modeling

In [ ]:
import os

from py2neo import Graph

# Connect to database
url = "bolt://localhost:7687"
username = os.getenv("NEO4J_USER")
password = os.getenv("NEO4J_PASSWORD_")
database = "dimensional-modelling"

graph = Graph(url, auth=(username, password), name=database)

In [ ]:
con_1 = "CREATE INDEX node_index_id FOR (n:Movie) on (n.id)"
con_2 = "CREATE INDEX node_index_title FOR (n:Movie) on (n.title)"
load_data_1 = """
LOAD CSV WITH HEADERS FROM 'file:///MovieRevenue.csv' AS line
CREATE (m:Movie {id:line.Rank, studio:line.Studio,
     title:line.Title, year:line.Year})
CREATE (r:RevenueFact {rank:line.Rank,
     worldwide:toFloat(line.Worldwide),
     domestic:toFloat(line.Domestic),
     overseas:toFloat(line.Overseas)})
CREATE (m)-[:HAS_REVENUE_FACT]->(r);
"""
load_data_2 = """
LOAD CSV WITH HEADERS FROM 'file:///MovieRatingGenerated.csv' AS line
MATCH (m:Movie {id:line.Rank})
CREATE (r:RatingFact {rating:toFloat(line.Rating)})
CREATE (m)-[:HAS_RATING_FACT]->(r);
"""

for q in [con_1, con_2, load_data_1, load_data_2]:
    res = graph.run(q)
    print(res)

In [ ]:
# Movies by worldwide revenue and rating, sorted by worldwide revenue desc
query = """
MATCH (revFact:RevenueFact)<--(m:Movie)-->(rateFact:RatingFact)
RETURN m.title AS Title, revFact.worldwide AS Worldwide, rateFact.rating AS Rating ORDER BY Worldwide DESC;
"""
# Studios by worldwide revenue, sorted by worldwide revenue desc
query_2 = """
MATCH (revFact:RevenueFact)<--(m:Movie)
WITH m.studio AS Studio, sum(revFact.worldwide) AS totalRev
RETURN Studio, totalRev as `Total Worldwide Revenue` ORDER BY totalRev DESC;
"""

## Get Schema
https://neo4j.com/developer/kb/viewing-schema-data-with-apoc/

In [ ]:
get_schema = """
CALL apoc.meta.schema() YIELD value as schemaMap
UNWIND keys(schemaMap) as label
WITH label, schemaMap[label] as data
WHERE data.type = "node"
UNWIND keys(data.properties) as property
WITH label, property, data.properties[property] as propData
RETURN label,
property,
propData.type as type,
propData.indexed as isIndexed,
propData.unique as uniqueConstraint,
propData.existence as existenceConstraint
"""
res = graph.run(get_schema).to_data_frame()
res